In [2]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione da ottimizzare CYLINDER()

In [3]:
function cylinder(radius=.5, height=2., angle=2*pi)
    function cylinder0(shape=[36, 1])
        V, CV = Lar.cuboidGrid(shape)
        CV = [[[u,v,w],[w,v,t]] for (u,v,w,t) in CV]
        CV = reduce(append!,CV)
        V = [angle/shape[1] 0.0 ; 0.0 1.0/shape[2]]*V
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v)=p;[radius*cos(u);radius*sin(u);
            height*v] end, W)...)
        W, CW = simplifyCells(V, CV)
        return W, CW
    end
    return cylinder0
end

cylinder (generic function with 4 methods)

In [4]:
@btime W,CW = Lar.cylinder()();

  269.276 μs (3382 allocations: 295.89 KiB)


In [5]:
@code_warntype Lar.cylinder()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#cylinder0#289"{Float64,Float64,Float64}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ─ %1 = Base.vect(36, 1)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Array,Array{Array{Int64,1},1}}
└──      return %2


Funzione NON è type unstable poiche Body::Tuple{Array,Array{Array{Int64,1},1}}


In [8]:
@benchmark Lar.cylinder()()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  269.975 μs …   7.416 ms  ┊ GC (min … max): 0.00% … 94.20%
 Time  (median):     280.867 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   343.451 μs ± 392.522 μs  ┊ GC (mean ± σ):  8.73% ±  7.43%

  █▇▆▅▄▃▃▂▂▁▁▁▁▁                           ▁▁▁                  ▂
  ██████████████████▇▇▇▇▇▆▆▆▆▆▆▅▆▅▆▅▆▅▆▆▅▆▅██████▇▆▆▅▅▄▅▆▄▃▄▅▄▄ █
  270 μs        Histogram: log(frequency) by time        711 μs <

 Memory estimate: 295.83 KiB, allocs estimate: 3380.

## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [9]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()

2

In [15]:
function cylinder2(radius=.5, height=2., angle=2*pi)
    function cylinder02(shape=[36, 1])
        V, CV = Lar.cuboidGrid(shape)
        CV = [[[u,v,w],[w,v,t]] for (u,v,w,t) in CV]
        CV = reduce(append!,CV)
        V = [angle/shape[1] 0.0 ; 0.0 1.0/shape[2]]*V
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
    end
        V = hcat(map(p->let(u, v)=p;[radius*cos(u);radius*sin(u);
            height*v] end, W)...)
        W, CW = Lar.simplifyCells(V, CV)
        return W, CW
    end
    return cylinder02
end

cylinder2 (generic function with 4 methods)

In [16]:
@btime W,CW = cylinder2()();

  250.571 μs (3117 allocations: 272.84 KiB)


Uso dei threads + refactoring del codice migliorano le prestazioni , non di molto ma sono migliori , poco altro da fare dato che la funzione si presta gia ben strutturata . Funzioni predefinite meglio non toccarle , con input grandi si rischierebbe di far scendere il livello di prestazioni , anche se in alcuni casi converebbe ridefinirle .